In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
from shared.spark_config import create_spark_config

from libs.configuration import configure

env = configure()
conf = create_spark_config("M2_Processors.airport.raw")

{'DATASTORAGE_AZURE_ACCESSKEY': 'dYj3PCGRyYHYR9pRo2xPa9BQ7t/sUonACJ29wcPdpa+1IC70rMbZmpfpHJJWJtk7OeASL+eQDIWq+AStGg3CCA==',
 'DATASTORAGE_AZURE_ACCOUNTNAME': 'icebergpe6t5cug7x9f45p8',
 'KAFKA_BOOTSTRAP_SERVERS': 'localhost:9092',
 'KAFKA_SCHEMA_REGISTRY_URL': 'http://localhost:8081',
 'KAFKA_SESSION_TIMEOUT_MS': '45000',
 'KAFKA_TOPIC_RAW_AIRCRAFT': 'raw.aircraft',
 'KAFKA_TOPIC_RAW_AIRLINE': 'raw.airline',
 'KAFKA_TOPIC_RAW_AIRPORT': 'raw.airport',
 'KAFKA_TOPIC_RAW_FLIGHT_SIGNAL': 'raw.flight_signal',
 'KAFKA_TOPIC_T1_AIRCRAFT': 't1.aircraft',
 'KAFKA_TOPIC_T1_AIRLINE': 't1.airline',
 'KAFKA_TOPIC_T1_AIRPORT': 't1.airport',
 'KAFKA_TOPIC_T1_FLIGHT_SIGNAL': 't1.flight_signal',
 'KAFKA_TOPIC_T2_AIRCRAFT': 't2.aircraft',
 'KAFKA_TOPIC_T2_AIRLINE': 't2.airline',
 'KAFKA_TOPIC_T2_AIRPORT': 't2.airport',
 'KAFKA_TOPIC_T2_FLIGHT_SIGNAL': 't2.flight_signal',
 'POSTGRE_JDBC_URL': 'jdbc:postgresql://localhost:5432/iceberg-catalog',
 'POSTGRE_PASSWORD': 'grebeci',
 'POSTGRE_USER': 'iceberg',
 

In [2]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [3]:
schema = T.StructType(
    [
        T.StructField("icao", T.StringType(), True),
        T.StructField("iata", T.StringType(), True),
        T.StructField("name", T.StringType(), True),
        T.StructField("city", T.StringType(), True),
        T.StructField("subd", T.StringType(), True),
        T.StructField("country", T.StringType(), True),
        T.StructField("elevation", T.FloatType(), True),
        T.StructField("lat", T.FloatType(), True),
        T.StructField("lon", T.FloatType(), True),
        T.StructField("tz", T.StringType(), True),
        T.StructField("lid", T.StringType(), True),
    ]
)
df = spark.read.csv(
    f"abfss://sampledata@{env.DATASTORAGE_AZURE_ACCOUNTNAME}.dfs.core.windows.net/airports.csv",
    schema,
    header=True,
    quote='"',
)
df.show()

+----+----+--------------------+------------+--------------+-------+---------+---------+-----------+-------------------+----+
|icao|iata|                name|        city|          subd|country|elevation|      lat|        lon|                 tz| lid|
+----+----+--------------------+------------+--------------+-------+---------+---------+-----------+-------------------+----+
|00AA|NULL|Aero B Ranch Airport|       Leoti|        Kansas|     US|   3435.0| 38.70402| -101.47391|    America/Chicago|00AA|
|00AK|NULL|        Lowell Field|Anchor Point|        Alaska|     US|    252.0| 59.94889| -151.69221|  America/Anchorage|00AK|
|00AL|NULL|        Epps Airpark|     Harvest|       Alabama|     US|    820.0| 34.86481|  -86.77028|    America/Chicago|00AL|
|00AR|NULL|      Arland Airport|  Bennington|        Kansas|     US|   1352.0| 38.96965| -97.601555|    America/Chicago|00AR|
|00AS|NULL|      Fulton Airport|        Alex|      Oklahoma|     US|   1100.0|34.942802| -97.818016|    America/Chicag

In [4]:
df = df.withColumn("created_ts", F.current_timestamp())
df.show()

+----+----+--------------------+------------+--------------+-------+---------+---------+-----------+-------------------+----+--------------------+--------------------+
|icao|iata|                name|        city|          subd|country|elevation|      lat|        lon|                 tz| lid|          created_ts|          updated_ts|
+----+----+--------------------+------------+--------------+-------+---------+---------+-----------+-------------------+----+--------------------+--------------------+
|00AA|NULL|Aero B Ranch Airport|       Leoti|        Kansas|     US|   3435.0| 38.70402| -101.47391|    America/Chicago|00AA|2024-01-20 21:20:...|2024-01-20 21:20:...|
|00AK|NULL|        Lowell Field|Anchor Point|        Alaska|     US|    252.0| 59.94889| -151.69221|  America/Anchorage|00AK|2024-01-20 21:20:...|2024-01-20 21:20:...|
|00AL|NULL|        Epps Airpark|     Harvest|       Alabama|     US|    820.0| 34.86481|  -86.77028|    America/Chicago|00AL|2024-01-20 21:20:...|2024-01-20 21:

In [7]:
df.writeTo("dev.raw.airports").append()